# 1. Initialization Code

## Install a Spark docker using the following commands.

In [ ]:
! docker pull bitnami/spark && \
docker network create spark_network && \
docker run -d --name spark --network=spark_network -e SPARK_MODE=master bitnami/spark
! ln -s /opt/conda/lib/libtinfo.so /opt/conda/lib/libtinfor.so.6
print('Done')

## Install pyspark.

In [ ]:
import pip

def install(package):
    if hasattr(pip, 'main'):
        pip.main(['install', package])
    else:
        pip._internal.main(['install', package])

install('pyspark')
        
print('Done')

## Initialize the Spark context variables.

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *

def initspark(appname = "Notebook", servername = "local[*]"):
    print ('initializing pyspark')
    conf = SparkConf().setAppName(appname).setMaster(servername)
    sc = SparkContext(conf=conf)
    spark = SparkSession.builder.appName(appname).enableHiveSupport().getOrCreate()
    sc.setLogLevel("ERROR")
    print ('pyspark initialized')
    return sc, spark, conf

sc, spark, conf = initspark()
print(sc, spark)

initializing pyspark
pyspark initialized
<SparkContext master=local[*] appName=Notebook> <pyspark.sql.session.SparkSession object at 0x7fbc44c16b90>


## Initialize helper functions to run Java inside cells.

In [183]:
# https://colab.research.google.com/github/apache/beam/blob/master/examples/notebooks/get-started/try-apache-beam-java.ipynb#scrollTo=CgTXBdTsBn1F
# Run and print a shell command.
def run(cmd, progress = True, verbose = False):
  if progress:
      print('>> {}'.format(cmd))
    
  if verbose:
      !{cmd}  # This is magic to run 'cmd' in the shell.
      print('')
  else:
      ! {cmd} > /dev/null 2>&1

import os

# Download the gradle source.
gradle_version = 'gradle-5.0'
gradle_path = f"/opt/{gradle_version}"
if not os.path.exists(gradle_path):
  run(f"wget -q -nc -O gradle.zip https://services.gradle.org/distributions/{gradle_version}-bin.zip")
  run('unzip -q -d /opt gradle.zip')
  run('rm -f gradle.zip')

# We're choosing to use the absolute path instead of adding it to the $PATH environment variable.
def gradle(args):
  run(f"{gradle_path}/bin/gradle --console=plain {args}")

gradle('-v')
print('Done')
        

## Definition for %%java Python magic cell function.

In [215]:
from IPython.core.magic import register_line_magic, register_cell_magic, register_line_cell_magic
@register_cell_magic
def java(line, cell):
    """
    Written by Joseph Gagliardo Jr.
    joegagliardo@gmail.com
    2021-12-22
    """
    text = """
plugins {
  // id 'idea'     // Uncomment for IntelliJ IDE
  // id 'eclipse'  // Uncomment for Eclipse IDE

  // Apply java plugin and make it a runnable application.
  id 'java'
  id 'application'

  // 'shadow' allows us to embed all the dependencies into a fat jar.
  id 'com.github.johnrengelman.shadow' version '4.0.3'
}

// This is the path of the main class, stored within ./src/main/java/
mainClassName = 'samples.quickstart.{class_name}'

// Declare the sources from which to fetch dependencies.
repositories {
  mavenCentral()
}

// Java version compatibility.
sourceCompatibility = 1.8
targetCompatibility = 1.8

// Use the latest Apache Beam major version 2.
// You can also lock into a minor version like '2.9.+'.
ext.apacheBeamVersion = '2.+'

// Declare the dependencies of the project.
dependencies {
  shadow "org.apache.beam:beam-sdks-java-core:$apacheBeamVersion"

  runtime "org.apache.beam:beam-runners-direct-java:$apacheBeamVersion"
  runtime "org.slf4j:slf4j-api:1.+"
  runtime "org.slf4j:slf4j-jdk14:1.+"

  testCompile "junit:junit:4.+"
}

// Configure 'shadowJar' instead of 'jar' to set up the fat jar.
shadowJar {
  baseName = '{class_name}' // Name of the fat jar file.
  classifier = null       // Set to null, otherwise 'shadow' appends a '-all' to the jar file name.
  manifest {
    attributes('Main-Class': mainClassName)  // Specify where the main class resides.
  }
}
"""   
    start = cell.find('class ')
    end = cell.find(' {')
    class_name = cell[start+6:end]
    progress = 'noprogress' not in line.lower()
    verbose = 'verbose' in line.lower()
    output = 'nooutput' not in line.lower()

    # if len(line) == 0:
    #     start = cell.find('class ')
    #     end = cell.find(' {')
    #     class_name = cell[start+6:end]
    # else:
    #     class_name = line
        
    
    run('rm src/main/java/samples/quickstart/*.java')
    run('rm build/libs/*.jar')
    run('rm -rf /tmp/outputs*', progress = progress, verbose = verbose)

    with open('build.gradle', 'w') as f:
        f.write(text.replace('{class_name}', class_name))

    with open(f'src/main/java/samples/quickstart/{class_name}.java', 'w') as f:
        f.write(cell)
        
    # Build the project.
    run(f"{gradle_path}/bin/gradle --console=plain build", progress = progress, verbose = verbose)
    run('ls -lh build/libs/', progress = progress, verbose = verbose)
    run(f"{gradle_path}/bin/gradle --console=plain runShadow", progress = progress, verbose = verbose)
    # run('head -n 20 /tmp/outputs*')
    if output:
        run('cat /tmp/outputs*', progress = False, verbose = True)

    print('Done')


# 2. Create allows you to upload data into a PCollection.

## Python

### Non Beam example of applying a map function to a collection.

In [229]:
x = ['one', 'two', 'three', 'four']
print(list(map(str.title, x)))

['One', 'Two', 'Three', 'Four']


### Simple transformation, turn the local collection into a PCollection and apply a Map PTransform on it.

In [230]:
import apache_beam as beam

with beam.Pipeline() as p:
    lines = (
        p | beam.Create(['one', 'two', 'three', 'four'])
          | beam.Map(str.title)
          | beam.Map(print)
    )

# lines is a PCollection object
print('lines = ', lines)


One
Two
Three
Four
lines =  PCollection[[230]: Map(print).None]


### Simple transformation using a lambda instead of a built in function.

In [231]:
import apache_beam as beam

with beam.Pipeline() as p:
    lines = (
        p | beam.Create(['one', 'two', 'three', 'four'])
          | beam.Map(lambda x : x.title())
          | beam.Map(print)
    )


One
Two
Three
Four


### Simple transformation using a user defined function.

In [232]:
import apache_beam as beam

def title(x):
    return x.title()

with beam.Pipeline() as p:
    lines = (
        p | beam.Create(['one', 'two', 'three', 'four'])
          | beam.Map(title)
          | beam.Map(print)
    )


One
Two
Three
Four


### Python the pipe `|` is actually just an operator overload to call the apply method of the pipeline. You would never do this in python, but it helps to understand what is going on under the hood.

In [226]:
import apache_beam as beam

with beam.Pipeline() as p:
        lines = ((p | beam.Create(['one', 'two', 'three', 'four']))
             .apply(beam.Map(str.title)) 
             .apply(beam.Map(print))
        )

One
Two
Three
Four


### The Spark equivalent would be to pload a local Python list into a Spark RDD and do a simple transformation.

In [233]:
rdd1 = ( sc.parallelize(['one', 'two', 'three', 'four'])
        
#           .map(str.title)
       )
rdd1.collect()


['one', 'two', 'three', 'four']

## Java

### Simple transformation using a lambda.


In [234]:
%%java
package samples.quickstart;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.values.PCollection;
import org.apache.beam.sdk.values.TypeDescriptors;
import org.apache.beam.sdk.transforms.Create;
import org.apache.beam.sdk.transforms.MapElements;
import org.apache.beam.sdk.io.TextIO;
import java.util.*;

public class Create1 {
    public static void main(String[] args) {

        String outputsPrefix = "/tmp/outputs";
        Pipeline p = Pipeline.create();
        
        PCollection<String> lines = p.apply(Create.of("one", "two", "three", "four"));
        lines = lines.apply(MapElements.into(TypeDescriptors.strings()).via((String line) -> line.toUpperCase()));
        lines.apply(TextIO.write().to(outputsPrefix));

        p.run().waitUntilFinish();
    }
}


>> rm src/main/java/samples/quickstart/*.java
>> rm build/libs/*.jar
>> rm -rf /tmp/outputs*
>> /opt/gradle-5.0/bin/gradle --console=plain build
>> ls -lh build/libs/
>> /opt/gradle-5.0/bin/gradle --console=plain runShadow
THREE
ONE
FOUR
TWO

Done


### Simple transformation using SimpleFunction instead of lambda.


In [217]:
%%java verbose
package samples.quickstart;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.values.PCollection;
import org.apache.beam.sdk.transforms.Create;
import org.apache.beam.sdk.transforms.MapElements;
import org.apache.beam.sdk.transforms.SimpleFunction;
import org.apache.beam.sdk.io.TextIO;
import java.util.*;

public class Create2 {
    public static void main(String[] args) {

        String outputsPrefix = "/tmp/outputs";
        Pipeline p = Pipeline.create();
        
        PCollection<String> lines = p.apply(Create.of("one", "two", "three", "four"));
        lines = lines.apply(MapElements.via(
            new SimpleFunction<String, String>() {
              @Override
              public String apply(String line) {
                return line.toUpperCase();
              }
            }));

        lines.apply("Write", TextIO.write().to(outputsPrefix));

        p.run().waitUntilFinish();
    }
}


>> rm src/main/java/samples/quickstart/*.java
>> rm build/libs/*.jar
>> rm -rf /tmp/outputs*

>> /opt/gradle-5.0/bin/gradle --console=plain build
> Task :compileJava
> Task :processResources NO-SOURCE
> Task :classes
> Task :jar
> Task :startScripts
> Task :distTar
> Task :distZip
> Task :shadowJar
> Task :startShadowScripts
> Task :shadowDistTar
> Task :shadowDistZip
> Task :assemble
> Task :compileTestJava NO-SOURCE
> Task :processTestResources NO-SOURCE
> Task :testClasses UP-TO-DATE
> Task :test NO-SOURCE
> Task :check UP-TO-DATE
> Task :build

BUILD SUCCESSFUL in 17s
9 actionable tasks: 9 executed

>> ls -lh build/libs/
total 43M
-rw-r--r-- 1 root root  43M Dec 23 18:32 Create2.jar
-rw-r--r-- 1 root root 2.1K Dec 23 18:32 Dataflowclass1.jar

>> /opt/gradle-5.0/bin/gradle --console=plain runShadow
> Task :compileJava UP-TO-DATE
> Task :processResources NO-SOURCE
> Task :classes UP-TO-DATE
> Task :shadowJar UP-TO-DATE
> Task :startShadowScripts UP-TO-DATE
> Task :installShadowDist



### Java simple transformation using SimpleFunction to wrap a User Defined Function.


In [227]:
%%java
package samples.quickstart;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.values.PCollection;
import org.apache.beam.sdk.transforms.Create;
import org.apache.beam.sdk.transforms.MapElements;
import org.apache.beam.sdk.transforms.SimpleFunction;
import org.apache.beam.sdk.io.TextIO;
import java.util.*;

public class Create3 {
    public static void main(String[] args) {

        String outputsPrefix = "/tmp/outputs";
        Pipeline p = Pipeline.create();
        
        PCollection<String> lines = p.apply(Create.of("one", "two", "three", "four"));
        lines = lines.apply(MapElements.via(
            new SimpleFunction<String, String>() {
              @Override
              public String apply(String line) {
                return upper(line);
              }
            }));

        lines.apply("Write", TextIO.write().to(outputsPrefix));

        p.run().waitUntilFinish();
    }
    
    public static String upper(String line) {
        return line.toUpperCase();
    }
}


>> rm src/main/java/samples/quickstart/*.java
>> rm build/libs/*.jar
>> rm -rf /tmp/outputs*
>> /opt/gradle-5.0/bin/gradle --console=plain build
>> ls -lh build/libs/
>> /opt/gradle-5.0/bin/gradle --console=plain runShadow
FOUR
TWO
THREE
ONE

Done


## Python read from CSV and use Map with lambda.
### Also it's a good idea to name the steps so it's easier to debug and monitor them later.


# 3. ReadFromText allows you to read a text file into a PCollection.

## Python


### It's a good idea to start naming the steps for debugging and monitoring later. Names must be unique in the pipeline.

In [235]:
! rm /tmp/outputs*

import apache_beam as beam
from apache_beam.io import ReadFromText, WriteToText

regionsfilename = 'regions.csv'
with beam.Pipeline() as p:
    regions = (
        p | 'Read' >> ReadFromText(regionsfilename)
          | 'Parse' >> beam.Map(lambda x : x.split(','))
          | 'Transform' >> beam.Map(lambda x : (int(x[0]), x[1].upper()))
          | 'Write' >> WriteToText('/tmp/outputs')
#          | 'Print' >> beam.Map(print)
    )
    #p.run() # implicit in Python when using with block

! cat /tmp/outputs*

(1, 'EASTERN')
(2, 'WESTERN')
(3, 'NORTHERN')
(4, 'SOUTHERN')


### Read from CSV and use ParDo

In [236]:
import apache_beam as beam
from apache_beam.io import ReadFromText, WriteToText

class RegionParseTuple(beam.DoFn):
    def process(self, element):
        regionid, regionname = element.split(',')
        #return [(int(regionid), regionname)] # ParDo's need to return a list
        yield (int(regionid), regionname) # Can also use yield instead of returning a list
#        yield (int(regionid), regionname.upper()) # Include a transformation instead of doing it as a separate step

regionsfilename = 'regions.csv'
with beam.Pipeline() as p:
    regions = (
        p | 'Read' >> ReadFromText(regionsfilename)
          | 'Parse' >> beam.ParDo(RegionParseTuple())
          #| 'Write' >> WriteToText('regions.out')
          | 'Print' >> beam.Map(print)
    )


(1, 'Eastern')
(2, 'Western')
(3, 'Northern')
(4, 'Southern')


## Java


### Read from CSV and use Map with lambda.

In [224]:
%%java
package samples.quickstart;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.values.PCollection;
import org.apache.beam.sdk.values.TypeDescriptors;
import org.apache.beam.sdk.transforms.MapElements;
import org.apache.beam.sdk.io.TextIO;

public class ReadRegions {
    public static void main(String[] args) {
        Pipeline p = Pipeline.create();

        String regionsInputFileName = "regions.csv";
        String outputsPrefix = "/tmp/outputs";

        PCollection<String> regions = p
            .apply("Read", TextIO.read().from(regionsInputFileName))
            .apply("Parse", MapElements.into(TypeDescriptors.strings()).via((String element) -> element.toUpperCase()));
        
        regions.apply(TextIO.write().to(outputsPrefix));
        p.run().waitUntilFinish();
    }
}


>> rm src/main/java/samples/quickstart/*.java
>> rm build/libs/*.jar
>> rm -rf /tmp/outputs*
>> /opt/gradle-5.0/bin/gradle --console=plain build
>> ls -lh build/libs/
>> /opt/gradle-5.0/bin/gradle --console=plain runShadow
3,NORTHERN
1,EASTERN
4,SOUTHERN
2,WESTERN

Done


### ParDo Example using anonymous class inline.

In [41]:
%%java
package samples.quickstart;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.values.PCollection;
import org.apache.beam.sdk.values.TypeDescriptors;
import org.apache.beam.sdk.io.TextIO;
import org.apache.beam.sdk.transforms.ParDo;
import org.apache.beam.sdk.transforms.DoFn;
import org.apache.beam.sdk.transforms.DoFn.ProcessContext;

public class Read1 {
    public static void main(String[] args) {
        Pipeline p = Pipeline.create();

        String regionsInputFileName = "regions.csv";
        String outputsPrefix = "/tmp/outputs";


        PCollection<String> regions = p
            .apply("Read", TextIO.read().from(regionsInputFileName))
            .apply("Parse", ParDo.of(new DoFn<String, String>() {
                @ProcessElement
                public void process(ProcessContext c) {
                    String element = c.element();
                    // String[] elements = element.split(",");
                    c.output(element + "*");
                }
            }));
        
        regions.apply(TextIO.write().to(outputsPrefix));
        p.run().waitUntilFinish();
    }
}



>> /opt/gradle-5.0/bin/gradle --console=plain build
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
> Task :compileJava
> Task :processResources NO-SOURCE
> Task :classes
> Task :jar
> Task :startScripts UP-TO-DATE
> Task :distTar
> Task :distZip
> Task :shadowJar
> Task :startShadowScripts
> Task :shadowDistTar
> Task :shadowDistZip
> Task :assemble
> Task :compileTestJava NO-SOURCE
> Task :processTestResources NO-SOURCE
> Task :testClasses UP-TO-DATE
> Task :test NO-SOURCE
> Task :check UP-TO-DATE
> Task :build

BUILD SUCCESSFUL in 17s
9 actionable tasks: 8 executed, 1 up-to-date

>> ls -lh build/libs/
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
total 212M
-rw-r--r-- 1 root root  43M Dec 23 06:38 Create1.jar
-rw-r--r-- 1 root root  43M Dec 23 07:02 Create2.jar
-rw-r--r-- 1 root root  43M Dec 23 14:26 Create3.jar
-rw-r--r-- 1 root root 4.0K Dec 23 14:35 Dataflowclass1.jar
-rw-r--r--

### ParDo using a defined class instead of an anonynous class.

In [44]:
%%java
package samples.quickstart;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.values.PCollection;
import org.apache.beam.sdk.values.TypeDescriptors;
import org.apache.beam.sdk.io.TextIO;
import org.apache.beam.sdk.transforms.ParDo;
import org.apache.beam.sdk.transforms.DoFn;
import org.apache.beam.sdk.transforms.DoFn.ProcessContext;

public class Read2 {
    public static void main(String[] args) {
        Pipeline p = Pipeline.create();

        String regionsInputFileName = "regions.csv";
        String outputsPrefix = "/tmp/outputs";


        PCollection<String> regions = p
            .apply("Read", TextIO.read().from(regionsInputFileName))
            .apply("Parse", ParDo.of(new AddStar()));
        
        regions.apply(TextIO.write().to(outputsPrefix));
        p.run().waitUntilFinish();
    }
    
    static class AddStar extends DoFn<String, String> {
        @ProcessElement
        public void process(@Element String line, OutputReceiver<String> out) {
            out.output(line + "*");
        }
    }
}



>> /opt/gradle-5.0/bin/gradle --console=plain build
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
> Task :compileJava
> Task :processResources NO-SOURCE
> Task :classes
> Task :jar
> Task :startScripts UP-TO-DATE
> Task :distTar
> Task :distZip
> Task :shadowJar
> Task :startShadowScripts
> Task :shadowDistTar
> Task :shadowDistZip
> Task :assemble
> Task :compileTestJava NO-SOURCE
> Task :processTestResources NO-SOURCE
> Task :testClasses UP-TO-DATE
> Task :test NO-SOURCE
> Task :check UP-TO-DATE
> Task :build

BUILD SUCCESSFUL in 17s
9 actionable tasks: 8 executed, 1 up-to-date

>> ls -lh build/libs/
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
total 254M
-rw-r--r-- 1 root root  43M Dec 23 06:38 Create1.jar
-rw-r--r-- 1 root root  43M Dec 23 07:02 Create2.jar
-rw-r--r-- 1 root root  43M Dec 23 14:26 Create3.jar
-rw-r--r-- 1 root root 5.8K Dec 23 14:46 Dataflowclass1.jar
-rw-r--r--

# 4. Parse into a model class.


## Python

### Create a model based on typing.NamedTuple so you can use properties instead of keys and use the Filter PTransform with lambda.

In [241]:
import apache_beam as beam
from apache_beam.io import ReadFromText, WriteToText
import typing

class Territory(typing.NamedTuple):
    territoryid: int
    territoryname: str
    regionid: int
beam.coders.registry.register_coder(Territory, beam.coders.RowCoder)
        
class TerritoryParseClass(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield Territory(int(territoryid), territoryname, int(regionid))

territoriesfilename = 'territories.csv'
with beam.Pipeline() as p:
    regions = (
        p | 'Read' >> ReadFromText(territoriesfilename)
          | 'Parse' >> beam.ParDo(TerritoryParseClass())
          | 'Filter 1' >> beam.Filter(lambda x : x.regionid % 2 == 0)
          | 'Filter 2' >> beam.Filter(lambda x : x.territoryname.startswith('S'))
          | 'Print' >> beam.Map(print)
#          | 'Write' >> WriteToText('regions.out')
    )


Territory(territoryid=31406, territoryname='Savannah', regionid=4)
Territory(territoryid=85251, territoryname='Scottsdale', regionid=2)
Territory(territoryid=90405, territoryname='Santa Monica', regionid=2)
Territory(territoryid=94105, territoryname='San Francisco', regionid=2)
Territory(territoryid=95054, territoryname='Santa Clara', regionid=2)
Territory(territoryid=95060, territoryname='Santa Cruz', regionid=2)
Territory(territoryid=98104, territoryname='Seattle', regionid=2)


### Use Filter with a UDF.

In [243]:
import apache_beam as beam
from apache_beam.io import ReadFromText, WriteToText
import typing

class Territory(typing.NamedTuple):
    territoryid: int
    territoryname: str
    regionid: int
beam.coders.registry.register_coder(Territory, beam.coders.RowCoder)
        
class TerritoryParseClass(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield Territory(int(territoryid), territoryname, int(regionid))

def startsWithS(element):
    return element.territoryname.startswith('S')

territoriesfilename = 'territories.csv'
with beam.Pipeline() as p:
    regions = (
        p | 'Read' >> ReadFromText(territoriesfilename)
          | 'Parse' >> beam.ParDo(TerritoryParseClass())
          | 'Filter' >> beam.Filter(startsWithS)
          | 'Print' >> beam.Map(print)
#          | 'Write' >> WriteToText('regions.out')
    )


Territory(territoryid=31406, territoryname='Savannah', regionid=4)
Territory(territoryid=48075, territoryname='Southfield', regionid=3)
Territory(territoryid=85251, territoryname='Scottsdale', regionid=2)
Territory(territoryid=90405, territoryname='Santa Monica', regionid=2)
Territory(territoryid=94105, territoryname='San Francisco', regionid=2)
Territory(territoryid=95054, territoryname='Santa Clara', regionid=2)
Territory(territoryid=95060, territoryname='Santa Cruz', regionid=2)
Territory(territoryid=98104, territoryname='Seattle', regionid=2)


### Use a ParDo class to accomplish filtering.

In [244]:
import apache_beam as beam
from apache_beam.io import ReadFromText, WriteToText
import typing

class Territory(typing.NamedTuple):
    territoryid: int
    territoryname: str
    regionid: int
beam.coders.registry.register_coder(Territory, beam.coders.RowCoder)
        
class TerritoryParseClass(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield Territory(int(territoryid), territoryname, int(regionid))

class StartsWithSFilter(beam.DoFn):
    def process(self, element):
        if element.territoryname.startswith('S'):
            yield element
            
territoriesfilename = 'territories.csv'
with beam.Pipeline() as p:
    regions = (
        p | 'Read' >> ReadFromText(territoriesfilename)
          | 'Parse' >> beam.ParDo(TerritoryParseClass())
          | 'Filter' >> beam.ParDo(StartsWithSFilter())
          | 'Print' >> beam.Map(print)
#          | 'Write' >> WriteToText('regions.out')
    )


Territory(territoryid=31406, territoryname='Savannah', regionid=4)
Territory(territoryid=48075, territoryname='Southfield', regionid=3)
Territory(territoryid=85251, territoryname='Scottsdale', regionid=2)
Territory(territoryid=90405, territoryname='Santa Monica', regionid=2)
Territory(territoryid=94105, territoryname='San Francisco', regionid=2)
Territory(territoryid=95054, territoryname='Santa Clara', regionid=2)
Territory(territoryid=95060, territoryname='Santa Cruz', regionid=2)
Territory(territoryid=98104, territoryname='Seattle', regionid=2)


### Put the parsing and filtering all into one ParDo.

In [246]:
import apache_beam as beam
from apache_beam.io import ReadFromText, WriteToText
import typing

class Territory(typing.NamedTuple):
    territoryid: int
    territoryname: str
    regionid: int
beam.coders.registry.register_coder(Territory, beam.coders.RowCoder)
        
class TerritoryParseClass(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        if territoryname.startswith('S'):
            yield Territory(int(territoryid), territoryname, int(regionid))

territoriesfilename = 'territories.csv'
with beam.Pipeline() as p:
    regions = (
        p | 'Read' >> ReadFromText(territoriesfilename)
          | 'Parse' >> beam.ParDo(TerritoryParseClass())
          | 'Print' >> beam.Map(print)
#          | 'Write' >> WriteToText('regions.out')
    )


Territory(territoryid=31406, territoryname='Savannah', regionid=4)
Territory(territoryid=48075, territoryname='Southfield', regionid=3)
Territory(territoryid=85251, territoryname='Scottsdale', regionid=2)
Territory(territoryid=90405, territoryname='Santa Monica', regionid=2)
Territory(territoryid=94105, territoryname='San Francisco', regionid=2)
Territory(territoryid=95054, territoryname='Santa Clara', regionid=2)
Territory(territoryid=95060, territoryname='Santa Cruz', regionid=2)
Territory(territoryid=98104, territoryname='Seattle', regionid=2)


## Java

### Parse a CSV into a class and filter it using a Pardo

In [239]:
%%java
package samples.quickstart;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.values.PCollection;
import org.apache.beam.sdk.values.TypeDescriptors;
import org.apache.beam.sdk.io.TextIO;
import org.apache.beam.sdk.transforms.ParDo;
import org.apache.beam.sdk.transforms.DoFn;
import org.apache.beam.sdk.transforms.Filter;
import org.apache.beam.sdk.transforms.DoFn.ProcessContext;
import org.apache.beam.sdk.coders.DefaultCoder;
import org.apache.beam.sdk.coders.AvroCoder;
import org.apache.beam.sdk.transforms.SerializableFunction;

import org.slf4j.Logger;
import org.slf4j.LoggerFactory;

public class ReadTerritories {
    public static void main(String[] args) {
        Pipeline p = Pipeline.create();

        String territoriesInputFileName = "territories.csv";
        String outputsPrefix = "/tmp/outputs";

        PCollection<Territory> territories = p
            .apply("Read", TextIO.read().from(territoriesInputFileName))
            .apply("Parse", ParDo.of(new ParseTerritories()))
            .apply("Filter", ParDo.of(new FilterTerritories()))
        ;                   
        
        territories.apply(TextIO.<Territory>writeCustomType().to(outputsPrefix).withFormatFunction(new SerializeTerritory()));
        p.run().waitUntilFinish();
    }
    
    @DefaultCoder(AvroCoder.class)
    static class Territory {
        Long territoryID;
        String territoryName;
        Long regionID;
        
        Territory() {}
        
        Territory(long territoryID, String territoryName, long regionID) {
            this.territoryID = territoryID;
            this.territoryName = territoryName;
            this.regionID = regionID;
        }
        
        @Override
        public String toString() {
            return String.format("(territoryID = %d, territoryName = %s, regionID = %d)", territoryID, territoryName, regionID);
        }

    }
    
    static class SerializeTerritory implements SerializableFunction<Territory, String> {
        @Override
        public String apply(Territory input) {
          return input.toString();
        }
    }

    static class ParseTerritories extends DoFn<String, Territory> {
        private static final Logger LOG = LoggerFactory.getLogger(ParseTerritories.class);

        @ProcessElement
        public void process(ProcessContext c) {
            String[] columns = c.element().split(",");
            try {
                Long territoryID = Long.parseLong(columns[0].trim());
                String territoryName = columns[1].trim();
                Long regionID = Long.parseLong(columns[2].trim());
                c.output(new Territory(territoryID, territoryName, regionID));
            } catch (ArrayIndexOutOfBoundsException | NumberFormatException e) {
                LOG.info("ParseTerritories: parse error on '" + c.element() + "': " + e.getMessage());
            }
        }
    }
    static class FilterTerritories extends DoFn<Territory, Territory> {
        private static final Logger LOG = LoggerFactory.getLogger(FilterTerritories.class);

        @ProcessElement
        public void process(@Element Territory t, OutputReceiver<Territory> o) {
            if (t.territoryID % 2 == 0 && t.territoryName.startsWith("S")) {
                o.output(t);
            }
        }
    }

}


>> rm src/main/java/samples/quickstart/*.java
>> rm build/libs/*.jar
>> rm -rf /tmp/outputs*
>> /opt/gradle-5.0/bin/gradle --console=plain build
>> ls -lh build/libs/
>> /opt/gradle-5.0/bin/gradle --console=plain runShadow
(territoryID = 31406, territoryName = Savannah, regionID = 4)
(territoryID = 95054, territoryName = Santa Clara, regionID = 2)
(territoryID = 95060, territoryName = Santa Cruz, regionID = 2)
(territoryID = 98104, territoryName = Seattle, regionID = 2)

Done


### Parse a CSV into a class and filter it using and anonymous class to create the condition.

In [247]:
%%java
package samples.quickstart;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.values.PCollection;
import org.apache.beam.sdk.values.TypeDescriptors;
import org.apache.beam.sdk.io.TextIO;
import org.apache.beam.sdk.transforms.ParDo;
import org.apache.beam.sdk.transforms.DoFn;
import org.apache.beam.sdk.transforms.Filter;
import org.apache.beam.sdk.transforms.DoFn.ProcessContext;
import org.apache.beam.sdk.coders.DefaultCoder;
import org.apache.beam.sdk.coders.AvroCoder;
import org.apache.beam.sdk.transforms.SerializableFunction;

import org.slf4j.Logger;
import org.slf4j.LoggerFactory;

public class ReadTerritories {
    public static void main(String[] args) {
        Pipeline p = Pipeline.create();

        String territoriesInputFileName = "territories.csv";
        String outputsPrefix = "/tmp/outputs";

        PCollection<Territory> territories = p
            .apply("Read", TextIO.read().from(territoriesInputFileName))
            .apply("Parse", ParDo.of(new ParseTerritories()))
            .apply("Filter", Filter.by(new SerializableFunction<Territory, Boolean>() {
                @Override
                public Boolean apply(Territory t) {
                    return t.territoryID % 2 == 0 && t.territoryName.startsWith("S");
                }
            }))
        ;                   
        
        territories.apply(TextIO.<Territory>writeCustomType().to(outputsPrefix).withFormatFunction(new SerializeTerritory()));
        p.run().waitUntilFinish();
    }
    
    @DefaultCoder(AvroCoder.class)
    static class Territory {
        Long territoryID;
        String territoryName;
        Long regionID;
        
        Territory() {}
        
        Territory(long territoryID, String territoryName, long regionID) {
            this.territoryID = territoryID;
            this.territoryName = territoryName;
            this.regionID = regionID;
        }
        
        @Override
        public String toString() {
            return String.format("(territoryID = %d, territoryName = %s, regionID = %d)", territoryID, territoryName, regionID);
        }

    }
    
    static class SerializeTerritory implements SerializableFunction<Territory, String> {
        @Override
        public String apply(Territory input) {
          return input.toString();
        }
    }

    static class ParseTerritories extends DoFn<String, Territory> {
        private static final Logger LOG = LoggerFactory.getLogger(ParseTerritories.class);

        @ProcessElement
        public void process(ProcessContext c) {
            String[] columns = c.element().split(",");
            try {
                Long territoryID = Long.parseLong(columns[0].trim());
                String territoryName = columns[1].trim();
                Long regionID = Long.parseLong(columns[2].trim());
                c.output(new Territory(territoryID, territoryName, regionID));
            } catch (ArrayIndexOutOfBoundsException | NumberFormatException e) {
                LOG.info("ParseTerritories: parse error on '" + c.element() + "': " + e.getMessage());
            }
        }
    }
    static class FilterTerritories extends DoFn<Territory, Territory> {
        private static final Logger LOG = LoggerFactory.getLogger(FilterTerritories.class);

        @ProcessElement
        public void process(@Element Territory t, OutputReceiver<Territory> o) {
            if (t.territoryID % 2 == 0 && t.territoryName.startsWith("S")) {
                o.output(t);
            }
        }
    }

}


>> rm src/main/java/samples/quickstart/*.java
>> rm build/libs/*.jar
>> rm -rf /tmp/outputs*
>> /opt/gradle-5.0/bin/gradle --console=plain build
>> ls -lh build/libs/
>> /opt/gradle-5.0/bin/gradle --console=plain runShadow
(territoryID = 95060, territoryName = Santa Cruz, regionID = 2)
(territoryID = 31406, territoryName = Savannah, regionID = 4)
(territoryID = 98104, territoryName = Seattle, regionID = 2)
(territoryID = 95054, territoryName = Santa Clara, regionID = 2)

Done


### Parse a CSV into a class and filter it in one step.

In [248]:
%%java
package samples.quickstart;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.values.PCollection;
import org.apache.beam.sdk.values.TypeDescriptors;
import org.apache.beam.sdk.io.TextIO;
import org.apache.beam.sdk.transforms.ParDo;
import org.apache.beam.sdk.transforms.DoFn;
import org.apache.beam.sdk.transforms.Filter;
import org.apache.beam.sdk.transforms.DoFn.ProcessContext;
import org.apache.beam.sdk.coders.DefaultCoder;
import org.apache.beam.sdk.coders.AvroCoder;
import org.apache.beam.sdk.transforms.SerializableFunction;

import org.slf4j.Logger;
import org.slf4j.LoggerFactory;

public class ReadTerritories {
    public static void main(String[] args) {
        Pipeline p = Pipeline.create();

        String territoriesInputFileName = "territories.csv";
        String outputsPrefix = "/tmp/outputs";

        PCollection<Territory> territories = p
            .apply("Read", TextIO.read().from(territoriesInputFileName))
            .apply("Parse", ParDo.of(new ParseTerritories()))
        ;                   
        
        territories.apply(TextIO.<Territory>writeCustomType().to(outputsPrefix).withFormatFunction(new SerializeTerritory()));
        p.run().waitUntilFinish();
    }
    
    @DefaultCoder(AvroCoder.class)
    static class Territory {
        Long territoryID;
        String territoryName;
        Long regionID;
        
        Territory() {}
        
        Territory(long territoryID, String territoryName, long regionID) {
            this.territoryID = territoryID;
            this.territoryName = territoryName;
            this.regionID = regionID;
        }
        
        @Override
        public String toString() {
            return String.format("(territoryID = %d, territoryName = %s, regionID = %d)", territoryID, territoryName, regionID);
        }

    }
    
    static class SerializeTerritory implements SerializableFunction<Territory, String> {
        @Override
        public String apply(Territory input) {
          return input.toString();
        }
    }

    static class ParseTerritories extends DoFn<String, Territory> {
        private static final Logger LOG = LoggerFactory.getLogger(ParseTerritories.class);

        @ProcessElement
        public void process(ProcessContext c) {
            String[] columns = c.element().split(",");
            try {
                Long territoryID = Long.parseLong(columns[0].trim());
                String territoryName = columns[1].trim();
                Long regionID = Long.parseLong(columns[2].trim());
                if (territoryName.startsWith("S")) {
                    c.output(new Territory(territoryID, territoryName, regionID));
                }
            } catch (ArrayIndexOutOfBoundsException | NumberFormatException e) {
                LOG.info("ParseTerritories: parse error on '" + c.element() + "': " + e.getMessage());
            }
        }
    }
    static class FilterTerritories extends DoFn<Territory, Territory> {
        private static final Logger LOG = LoggerFactory.getLogger(FilterTerritories.class);

        @ProcessElement
        public void process(@Element Territory t, OutputReceiver<Territory> o) {
            if (t.territoryID % 2 == 0 && t.territoryName.startsWith("S")) {
                o.output(t);
            }
        }
    }

}


>> rm src/main/java/samples/quickstart/*.java
>> rm build/libs/*.jar
>> rm -rf /tmp/outputs*
>> /opt/gradle-5.0/bin/gradle --console=plain build
>> ls -lh build/libs/
>> /opt/gradle-5.0/bin/gradle --console=plain runShadow
(territoryID = 48075, territoryName = Southfield, regionID = 3)
(territoryID = 90405, territoryName = Santa Monica, regionID = 2)
(territoryID = 94105, territoryName = San Francisco, regionID = 2)
(territoryID = 31406, territoryName = Savannah, regionID = 4)
(territoryID = 95054, territoryName = Santa Clara, regionID = 2)
(territoryID = 95060, territoryName = Santa Cruz, regionID = 2)
(territoryID = 85251, territoryName = Scottsdale, regionID = 2)
(territoryID = 98104, territoryName = Seattle, regionID = 2)

Done


# 5. Create multiple outputs from a single read.

## Python

### Send the same data down multiple paths, such as to group it on two different keys with one read from the source.

In [266]:
import apache_beam as beam
from apache_beam import pvalue
from apache_beam.io import ReadFromText, WriteToText
import typing

class Territory(typing.NamedTuple):
    territoryid: int
    territoryname: str
    regionid: int
beam.coders.registry.register_coder(Territory, beam.coders.RowCoder)
        
class TerritoryParseClass(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield Territory(int(territoryid), territoryname, int(regionid))

territoriesfilename = 'territories.csv'
with beam.Pipeline() as p:
    territories = (p 
                    | 'Read' >> ReadFromText(territoriesfilename) 
                    | 'Parse' >> beam.ParDo(TerritoryParseClass())
                  )

    # Branch 1
    (territories 
         | 'Lowercase' >> beam.Map(lambda x : (x.territoryid, x.territoryname.lower(), x.regionid))
         | 'Write Lower' >> WriteToText('/tmp/territories_lower.out')
    )
    
    # Branch 2
    (territories 
         | 'Uppercase' >> beam.Map(lambda x : (x.territoryid, x.territoryname.upper(), x.regionid))
         | 'Write Upper' >> WriteToText('/tmp/territories_upper.out')
    )

! echo "Lower" && cat /tmp/territories_lower.out* && echo "Upper" && cat /tmp/territories_upper.out*
    

Lower
(1730, 'bedford', 1)
(1581, 'westboro', 1)
(1833, 'georgetow', 1)
(2116, 'boston', 1)
(2139, 'cambridge', 1)
(2184, 'braintree', 1)
(2903, 'providence', 1)
(3049, 'hollis', 3)
(3801, 'portsmouth', 3)
(6897, 'wilton', 1)
(7960, 'morristown', 1)
(8837, 'edison', 1)
(10019, 'new york', 1)
(10038, 'new york', 1)
(11747, 'mellvile', 1)
(14450, 'fairport', 1)
(19428, 'philadelphia', 3)
(19713, 'neward', 1)
(20852, 'rockville', 1)
(27403, 'greensboro', 1)
(27511, 'cary', 1)
(29202, 'columbia', 4)
(30346, 'atlanta', 4)
(31406, 'savannah', 4)
(32859, 'orlando', 4)
(33607, 'tampa', 4)
(40222, 'louisville', 1)
(44122, 'beachwood', 3)
(45839, 'findlay', 3)
(48075, 'southfield', 3)
(48084, 'troy', 3)
(48304, 'bloomfield hills', 3)
(53404, 'racine', 3)
(55113, 'roseville', 3)
(55439, 'minneapolis', 3)
(60179, 'hoffman estates', 2)
(60601, 'chicago', 2)
(72716, 'bentonville', 4)
(75234, 'dallas', 4)
(78759, 'austin', 4)
(80202, 'denver', 2)
(80909, 'colorado springs', 2)
(85014, 'phoenix', 2)
(

### Use TaggedOutput in the ParDo to split data into two different paths with different data on each.

In [268]:
import apache_beam as beam
from apache_beam import pvalue
from apache_beam.io import ReadFromText, WriteToText
import typing

class Territory(typing.NamedTuple):
    territoryid: int
    territoryname: str
    regionid: int
beam.coders.registry.register_coder(Territory, beam.coders.RowCoder)
        
class OddEvenTerritoryParseClass(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        if int(regionid) % 2 == 0:
            yield pvalue.TaggedOutput('Even', Territory(int(territoryid), territoryname, int(regionid)))
        else:
            yield pvalue.TaggedOutput('Odd', Territory(int(territoryid), territoryname, int(regionid)))

territoriesfilename = 'territories.csv'
with beam.Pipeline() as p:
    lines = p | 'Read' >> ReadFromText(territoriesfilename) 
    # lines would return a tuple of the two tagged outputs
    # unpack the two outputs to two separate variables to process differently
    evens, odds = lines | 'Parse' >> beam.ParDo(OddEvenTerritoryParseClass()).with_outputs("Even", "Odd")
    
    evens | 'Write Even' >> WriteToText('/tmp/territories_even.out')
    
    odds | 'Write Odd' >> WriteToText('/tmp/territories_odd.out')

! echo "Evens" && cat /tmp/territories_even.out* && echo "Odds" && cat /tmp/territories_odd.out*

Evens
Territory(territoryid=29202, territoryname='Columbia', regionid=4)
Territory(territoryid=30346, territoryname='Atlanta', regionid=4)
Territory(territoryid=31406, territoryname='Savannah', regionid=4)
Territory(territoryid=32859, territoryname='Orlando', regionid=4)
Territory(territoryid=33607, territoryname='Tampa', regionid=4)
Territory(territoryid=60179, territoryname='Hoffman Estates', regionid=2)
Territory(territoryid=60601, territoryname='Chicago', regionid=2)
Territory(territoryid=72716, territoryname='Bentonville', regionid=4)
Territory(territoryid=75234, territoryname='Dallas', regionid=4)
Territory(territoryid=78759, territoryname='Austin', regionid=4)
Territory(territoryid=80202, territoryname='Denver', regionid=2)
Territory(territoryid=80909, territoryname='Colorado Springs', regionid=2)
Territory(territoryid=85014, territoryname='Phoenix', regionid=2)
Territory(territoryid=85251, territoryname='Scottsdale', regionid=2)
Territory(territoryid=90405, territoryname='Santa

## Java

### Send the same output down two different paths.

In [281]:
! rm /tmp/territories*

In [282]:
%%java nooutput
package samples.quickstart;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.values.PCollection;
import org.apache.beam.sdk.values.TypeDescriptors;
import org.apache.beam.sdk.io.TextIO;
import org.apache.beam.sdk.transforms.ParDo;
import org.apache.beam.sdk.transforms.DoFn;
import org.apache.beam.sdk.transforms.Filter;
import org.apache.beam.sdk.transforms.DoFn.ProcessContext;
import org.apache.beam.sdk.coders.DefaultCoder;
import org.apache.beam.sdk.coders.AvroCoder;
import org.apache.beam.sdk.transforms.SerializableFunction;
import org.apache.beam.sdk.values.TupleTag;
import org.apache.beam.sdk.values.PCollectionTuple;
import org.apache.beam.sdk.values.TupleTagList;

import org.slf4j.Logger;
import org.slf4j.LoggerFactory;

public class ReadTerritories {

    public static void main(String[] args) {
        Pipeline p = Pipeline.create();

        String territoriesInputFileName = "territories.csv";
        String outputsPrefix = "/tmp/outputs";

        PCollection<Territory> territories = p
            .apply("Read", TextIO.read().from(territoriesInputFileName))
            .apply("Parse Territory", ParDo.of(new ParseTerritories()))
        ;                   
        
            
        territories
            .apply("Upper", ParDo.of(new DoFn<Territory, Territory>() {
                @ProcessElement
                public void process(ProcessContext c) {
                    Territory t = c.element();
                    c.output(new Territory(t.territoryID, t.territoryName.toUpperCase(), t.regionID));
                }
            }))
             .apply(TextIO.<Territory>writeCustomType().to("/tmp/territories_upper").withFormatFunction(new SerializeTerritory()));

        territories
            .apply("Lower", ParDo.of(new DoFn<Territory, Territory>() {
                @ProcessElement
                public void process(ProcessContext c) {
                    Territory t = c.element();
                    c.output(new Territory(t.territoryID, t.territoryName.toLowerCase(), t.regionID));
                }
            }))
             .apply(TextIO.<Territory>writeCustomType().to("/tmp/territories_lower").withFormatFunction(new SerializeTerritory()));

        
        p.run().waitUntilFinish();
    }
    
    @DefaultCoder(AvroCoder.class)
    static class Territory {
        Long territoryID;
        String territoryName;
        Long regionID;
        
        Territory() {}
        
        Territory(long territoryID, String territoryName, long regionID) {
            this.territoryID = territoryID;
            this.territoryName = territoryName;
            this.regionID = regionID;
        }
        
        @Override
        public String toString() {
            return String.format("(territoryID = %d, territoryName = %s, regionID = %d)", territoryID, territoryName, regionID);
        }

    }
    
    static class SerializeTerritory implements SerializableFunction<Territory, String> {
        @Override
        public String apply(Territory input) {
          return input.toString();
        }
    }

    static class ParseTerritories extends DoFn<String, Territory> {
        private static final Logger LOG = LoggerFactory.getLogger(ParseTerritories.class);

        @ProcessElement
        public void process(ProcessContext c) {
            String[] columns = c.element().split(",");
            try {
                Long territoryID = Long.parseLong(columns[0].trim());
                String territoryName = columns[1].trim();
                Long regionID = Long.parseLong(columns[2].trim());
                c.output(new Territory(territoryID, territoryName, regionID));
            } catch (ArrayIndexOutOfBoundsException | NumberFormatException e) {
                LOG.info("ParseTerritoriesOddEvenSplit: parse error on '" + c.element() + "': " + e.getMessage());
            }
        }
    }

}


>> rm src/main/java/samples/quickstart/*.java
>> rm build/libs/*.jar
>> rm -rf /tmp/outputs*
>> /opt/gradle-5.0/bin/gradle --console=plain build
>> ls -lh build/libs/
>> /opt/gradle-5.0/bin/gradle --console=plain runShadow
Done


In [283]:
! echo "Upper" && cat /tmp/territories_upper* && echo "Lower" && cat /tmp/territories_lower*


Upper
(territoryID = 3049, territoryName = HOLLIS, regionID = 3)
(territoryID = 7960, territoryName = MORRISTOWN, regionID = 1)
(territoryID = 80909, territoryName = COLORADO SPRINGS, regionID = 2)
(territoryID = 90405, territoryName = SANTA MONICA, regionID = 2)
(territoryID = 95008, territoryName = CAMPBELL, regionID = 2)
(territoryID = 98004, territoryName = BELLEVUE, regionID = 2)
(territoryID = 1833, territoryName = GEORGETOW, regionID = 1)
(territoryID = 2184, territoryName = BRAINTREE, regionID = 1)
(territoryID = 11747, territoryName = MELLVILE, regionID = 1)
(territoryID = 19713, territoryName = NEWARD, regionID = 1)
(territoryID = 27511, territoryName = CARY, regionID = 1)
(territoryID = 31406, territoryName = SAVANNAH, regionID = 4)
(territoryID = 40222, territoryName = LOUISVILLE, regionID = 1)
(territoryID = 48075, territoryName = SOUTHFIELD, regionID = 3)
(territoryID = 53404, territoryName = RACINE, regionID = 3)
(territoryID = 60179, territoryName = HOFFMAN ESTATES, reg

### Use TupleTag to split the output into two separate path.

In [284]:
! rm /tmp/territories*

In [285]:
%%java nooutput
package samples.quickstart;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.values.PCollection;
import org.apache.beam.sdk.values.TypeDescriptors;
import org.apache.beam.sdk.io.TextIO;
import org.apache.beam.sdk.transforms.ParDo;
import org.apache.beam.sdk.transforms.DoFn;
import org.apache.beam.sdk.transforms.Filter;
import org.apache.beam.sdk.transforms.DoFn.ProcessContext;
import org.apache.beam.sdk.coders.DefaultCoder;
import org.apache.beam.sdk.coders.AvroCoder;
import org.apache.beam.sdk.transforms.SerializableFunction;
import org.apache.beam.sdk.values.TupleTag;
import org.apache.beam.sdk.values.PCollectionTuple;
import org.apache.beam.sdk.values.TupleTagList;

import org.slf4j.Logger;
import org.slf4j.LoggerFactory;

public class ReadTerritories {

    final static TupleTag<Territory> evenTag = new TupleTag<Territory>() {};
    final static TupleTag<Territory> oddTag = new TupleTag<Territory>() {};

    public static void main(String[] args) {
        Pipeline p = Pipeline.create();

        String territoriesInputFileName = "territories.csv";
        String outputsPrefix = "/tmp/outputs";

        PCollectionTuple territories = p
            .apply("Read", TextIO.read().from(territoriesInputFileName))
            .apply("OddEvenSplit", ParDo.of(new ParseTerritoriesOddEvenSplit()).withOutputTags(evenTag, TupleTagList.of(oddTag)))
        ;                   
        
        PCollection<Territory> evenTerritories = territories.get(evenTag);
        evenTerritories.apply(TextIO.<Territory>writeCustomType().to(outputsPrefix + "_even").withFormatFunction(new SerializeTerritory()));

        PCollection<Territory> oddTerritories = territories.get(oddTag);
        oddTerritories.apply(TextIO.<Territory>writeCustomType().to(outputsPrefix + "_odd").withFormatFunction(new SerializeTerritory()));
        p.run().waitUntilFinish();
    }
    
    @DefaultCoder(AvroCoder.class)
    static class Territory {
        Long territoryID;
        String territoryName;
        Long regionID;
        
        Territory() {}
        
        Territory(long territoryID, String territoryName, long regionID) {
            this.territoryID = territoryID;
            this.territoryName = territoryName;
            this.regionID = regionID;
        }
        
        @Override
        public String toString() {
            return String.format("(territoryID = %d, territoryName = %s, regionID = %d)", territoryID, territoryName, regionID);
        }

    }
    
    static class SerializeTerritory implements SerializableFunction<Territory, String> {
        @Override
        public String apply(Territory input) {
          return input.toString();
        }
    }

    static class ParseTerritoriesOddEvenSplit extends DoFn<String, Territory> {
        private static final Logger LOG = LoggerFactory.getLogger(ParseTerritoriesOddEvenSplit.class);

        @ProcessElement
        public void process(ProcessContext c) {


            String[] columns = c.element().split(",");
            try {
                Long territoryID = Long.parseLong(columns[0].trim());
                String territoryName = columns[1].trim();
                Long regionID = Long.parseLong(columns[2].trim());
                if (regionID % 2 == 0) {
                    c.output(evenTag, new Territory(territoryID, territoryName, regionID));
                } else {
                    c.output(oddTag, new Territory(territoryID, territoryName, regionID));
                }
            } catch (ArrayIndexOutOfBoundsException | NumberFormatException e) {
                LOG.info("ParseTerritoriesOddEvenSplit: parse error on '" + c.element() + "': " + e.getMessage());
            }
        }
    }

}


>> rm src/main/java/samples/quickstart/*.java
>> rm build/libs/*.jar
>> rm -rf /tmp/outputs*
>> /opt/gradle-5.0/bin/gradle --console=plain build
>> ls -lh build/libs/
>> /opt/gradle-5.0/bin/gradle --console=plain runShadow
Done


In [286]:
! echo "Odd" && cat /tmp/outputs_odd* && echo "Even" && cat /tmp/outputs_even*


Odd
(territoryID = 53404, territoryName = Racine, regionID = 3)
(territoryID = 14450, territoryName = Fairport, regionID = 1)
(territoryID = 27511, territoryName = Cary, regionID = 1)
(territoryID = 1581, territoryName = Westboro, regionID = 1)
(territoryID = 2903, territoryName = Providence, regionID = 1)
(territoryID = 8837, territoryName = Edison, regionID = 1)
(territoryID = 45839, territoryName = Findlay, regionID = 3)
(territoryID = 55113, territoryName = Roseville, regionID = 3)
(territoryID = 19428, territoryName = Philadelphia, regionID = 3)
(territoryID = 40222, territoryName = Louisville, regionID = 1)
(territoryID = 1833, territoryName = Georgetow, regionID = 1)
(territoryID = 3049, territoryName = Hollis, regionID = 3)
(territoryID = 10019, territoryName = New York, regionID = 1)
(territoryID = 48075, territoryName = Southfield, regionID = 3)
(territoryID = 55439, territoryName = Minneapolis, regionID = 3)
(territoryID = 19713, territoryName = Neward, regionID = 1)
(territ

# 6. To use Group, Join, Sort you need to reshape the data into a KV pair first.

## Python

## GroupByKey will cluster the elements as a list under each unique key. The data must be in a KV tuple pair first.

In [288]:
import apache_beam as beam
from apache_beam.io import ReadFromText

class Territory(typing.NamedTuple):
    territoryid: int
    territoryname: str
    regionid: int
beam.coders.registry.register_coder(Territory, beam.coders.RowCoder)
        
class TerritoryParseClass(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield Territory(int(territoryid), territoryname, int(regionid))

territoriesfilename = 'territories.csv'
with beam.Pipeline() as p:
    territories = (
                  p | 'Read Territories' >> ReadFromText('territories.csv')
                    | 'Parse Territories' >> beam.ParDo(TerritoryParseClass())
                    | 'Territories With Keys' >> beam.util.WithKeys(lambda x : x.regionid)
#                    | 'Group Territories' >> beam.GroupByKey() 
                    | 'Print Territories' >> beam.Map(print)
                  )


(1, Territory(territoryid=1730, territoryname='Bedford', regionid=1))
(1, Territory(territoryid=1581, territoryname='Westboro', regionid=1))
(1, Territory(territoryid=1833, territoryname='Georgetow', regionid=1))
(1, Territory(territoryid=2116, territoryname='Boston', regionid=1))
(1, Territory(territoryid=2139, territoryname='Cambridge', regionid=1))
(1, Territory(territoryid=2184, territoryname='Braintree', regionid=1))
(1, Territory(territoryid=2903, territoryname='Providence', regionid=1))
(3, Territory(territoryid=3049, territoryname='Hollis', regionid=3))
(3, Territory(territoryid=3801, territoryname='Portsmouth', regionid=3))
(1, Territory(territoryid=6897, territoryname='Wilton', regionid=1))
(1, Territory(territoryid=7960, territoryname='Morristown', regionid=1))
(1, Territory(territoryid=8837, territoryname='Edison', regionid=1))
(1, Territory(territoryid=10019, territoryname='New York', regionid=1))
(1, Territory(territoryid=10038, territoryname='New York', regionid=1))
(1, 

In [293]:
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam import coders
from apache_beam.transforms.sql import SqlTransform

import typing
import json

with beam.Pipeline() as p:
    parent = (
            p | 'Create Parent' >> beam.Create([(1, 'One'), (2, 'Two'), (4, 'Four')])
              | 'Map Parent' >> beam.Map(lambda x : beam.Row(parent_id = x[0], parent_name = x[1]))
    )

    child = (
            p | 'Create Child' >> beam.Create([('Uno', 1), ('Due', 2), ('Eins', 1), ('Una', 1), ('Dos', 2), ('Tres', 3)])
              | 'Map Child' >> beam.Map(lambda x : beam.Row(child_name = x[0], parent_id = x[1]))
    )
    
    result = ( {'parent': parent, 'child' : child} 
         | SqlTransform("""
             SELECT p.parent_id, p.parent_name, c.child_name 
             FROM parent as p 
             INNER JOIN child as c ON p.parent_id = c.parent_id
             """)
        | 'Map Join Output' >> beam.Map(lambda x : f'{x.parent_id} {x.parent_name} {x.child_name}')
        | 'Print Join' >> beam.Map(print)
        )


1 One Uno
1 One Eins
1 One Una
2 Two Due
2 Two Dos


In [297]:
import apache_beam as beam
from apache_beam import coders
from apache_beam.transforms.sql import SqlTransform

import typing

class Parent(typing.NamedTuple):
    parent_id: int
    parent_name: str
beam.coders.registry.register_coder(Parent, beam.coders.RowCoder)

class Child(typing.NamedTuple):
    child_name: str
    parent_id: int
beam.coders.registry.register_coder(Child, beam.coders.RowCoder)
        

with beam.Pipeline() as p:
    parent = (
            p | 'Create Parent' >> beam.Create([(1, 'One'), (2, 'Two')])
              | 'Map Parent' >> beam.Map(lambda x : Parent(parent_id = x[0], parent_name = x[1])).with_output_types(Parent)
              | 'Print 1' >> beam.Map(print)
    )

    child = (
            p | 'Create Child' >> beam.Create([('Uno', 1), ('Due', 2), ('Eins', 1), ('Una', 1), ('Dos', 2)])
              | 'Map Child' >> beam.Map(lambda x : Child(child_name = x[0], parent_id = x[1])).with_output_types(Child)
              | 'SQL Child' >> SqlTransform("""SELECT parent_id, count(*) as cnt from PCOLLECTION GROUP BY parent_id""")
              | 'Map for Print 2' >> beam.Map(lambda x : f'Parent {x.parent_id} count = {x.cnt}')
              | 'Print 2' >> beam.Map(print)
    )

Parent(parent_id=1, parent_name='One')
Parent(parent_id=2, parent_name='Two')
Parent 1 count = 3
Parent 2 count = 2


## Java